<a href="https://colab.research.google.com/github/Alexandre77777/data_mining/blob/main/9.%20%D0%9F%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%20%D0%B8%D1%81%D1%82%D0%BE%D1%80%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D1%85%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B8%20%D0%B2%D0%B8%D0%B7%D1%83%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F%20%D0%BF%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%BE%D0%B2%20%D0%BD%D0%B0%20%D0%BA%D0%B0%D1%80%D1%82%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B5%2C%20%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D1%83%D1%8F%20%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B%20%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE%20%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F/9_%D0%9F%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B8%D1%81%D1%82%D0%BE%D1%80%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D1%85_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B8_%D0%B2%D0%B8%D0%B7%D1%83%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%BF%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%BE%D0%B2_%D0%BD%D0%B0_%D0%BA%D0%B0%D1%80%D1%82%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B5%2C_%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D1%83%D1%8F_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

- **Используя известные нам методы машинного обучения, спрогнозируйте изменение динамики ряда данных, используя соответствующий датасет из предыдущей работы (численность насерения, показатель ВВП, средней продолжительности жизни и т.д).**

- **Визуализируйте прогноз на интерактивной картограмме (см. прошлую работу)**

Пример картограммы:

In [6]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go 

import plotly.express as px
df = px.data.gapminder()

# Список переменных для предсказания
target_variables = ['pop', 'lifeExp', 'gdpPercap']

# Выбранная страна для рассмотрения
country_of_interest = 'China'

for target_variable in target_variables:
    country_data = df[df['country'] == country_of_interest]

    # Разбивка датасета на тренировочный и тестовый
    train_size = int(len(country_data) * 0.8)
    train, test = country_data[:train_size], country_data[train_size:]

    # SARIMAX model
    model = SARIMAX(train[target_variable], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)

    # Генерация предсказаний
    predictions = model_fit.predict(start=len(train), end=len(train) + len(test) - 1, dynamic=False)

    # СКО
    mse = mean_squared_error(test[target_variable], predictions)
    print(f'Mean Squared Error for {target_variable}: {mse}')

    # Интерактивные карты
    combined_data = pd.concat([train, test])
    fig = px.choropleth(
        combined_data,
        locations="iso_alpha",
        color=target_variable,
        hover_name="country",
        animation_frame="year",
        color_continuous_scale=px.colors.sequential.Plasma,
        projection="natural earth",
        title=f'{target_variable} Forecast for {country_of_interest}'
    )

    fig.add_trace(go.Scattergeo(
        lon=[country_data['iso_alpha'].iloc[-1]],
        lat=[country_data['iso_alpha'].iloc[-1]],
        text=[f'Forecast: {predictions.iloc[-1]:,.2f}'],
        mode='text',
    ))

    fig.show()

Mean Squared Error for pop: 1.3730153935635072e+16


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-pac

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.



Mean Squared Error for lifeExp: 10.799127000000025


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning:

Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-pac

Mean Squared Error for gdpPercap: 4485025.087966897


# Задание 2.

-  **Используя известные нам методы машинного обучения, спрогнозируйте изменение динамики ряда данных с региональной привязкой, используя найденный Вами датасет в сети Интернет.**

- **Количество элементов ряде данных должно превышать 100. (динамика заболеваний по месяцам за несколько десятилетий, курс валюты и тд, чем больше данных для обучения, тем лучше.)**

- **Визуализируйте прогноз на интерактивной картограмме**


In [7]:
df = pd.read_csv('country_vaccinations.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86512 entries, 0 to 86511
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              86512 non-null  object 
 1   iso_code                             86512 non-null  object 
 2   date                                 86512 non-null  object 
 3   total_vaccinations                   43607 non-null  float64
 4   people_vaccinated                    41294 non-null  float64
 5   people_fully_vaccinated              38802 non-null  float64
 6   daily_vaccinations_raw               35362 non-null  float64
 7   daily_vaccinations                   86213 non-null  float64
 8   total_vaccinations_per_hundred       43607 non-null  float64
 9   people_vaccinated_per_hundred        41294 non-null  float64
 10  people_fully_vaccinated_per_hundred  38802 non-null  float64
 11  daily_vaccinations_per_milli

In [13]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.express as px

country_data.set_index('date', inplace=True)

# Колонка для предсказания
selected_columns = ['date', 'daily_vaccinations']

# Пустой датафрейм для предсказаний для каждой страны
all_forecasts = pd.DataFrame()

for country_of_interest in df['country'].unique():
    country_data = df[df['country'] == country_of_interest][selected_columns]

    # Конвертация даты в единной формат datetime
    country_data['date'] = pd.to_datetime(country_data['date'])

    # Пустые значения заполняем нулями
    country_data['daily_vaccinations'].fillna(0, inplace=True)

    # Разбивка датасета на тренировачный и тестовый
    train_size = int(len(country_data) * 0.8)
    train, test = country_data[:train_size], country_data[train_size:]

    # SARIMAX model
    try:
        model = SARIMAX(train['daily_vaccinations'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
        model_fit = model.fit(disp=False)

        # Предсказания
        predictions = model_fit.predict(start=len(train), end=len(train) + len(test) - 1, dynamic=False)

        forecast = test.copy()
        forecast['predicted'] = predictions
        forecast['country'] = country_of_interest
        all_forecasts = pd.concat([all_forecasts, forecast])
    except Exception as e:
        print(f"Error fitting SARIMAX for {country_of_interest}: {e}")

fig = px.line(all_forecasts, x='date', y=['daily_vaccinations', 'predicted'], color='country',
              labels={'value': 'Daily Vaccinations'},
              title=f'Daily Vaccinations Forecast for All Countries using SARIMAX')

fig.show()


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodel

In [15]:
all_forecasts = pd.merge(all_forecasts, df[['country', 'iso_code']], on='country', how='left')

In [ ]:
fig_map = px.choropleth(
    all_forecasts,
    locations="iso_code",
    locationmode="ISO-3",
    color="predicted", 
    hover_name="country",
    animation_frame="date",
    color_continuous_scale=px.colors.sequential.Plasma,
    projection="natural earth",
    title='Daily Vaccinations Forecast for All Countries using SARIMAX'
)

fig_map.show()